### Installation

In [ ]:
!pip install unsloth


### Model Load

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.5.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


<a name="Data"></a>
### Data Prep
Convert the DataSet into Alpaca Format

In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd


# Define the Alpaca prompt template exactly as in the original code
alpaca_prompt = """You are a security expert helping developers identify vulnerable lines of code.

### Task:
Identify the exact line in the following {language} code where the {cwe_id} vulnerability occurs and explain briefly.

### Code:
{code_snippet}

### Output:
{vulnerable_line}
Description: [{cwe_id}] {description}{eos}
"""

# Define the EOS_TOKEN
# EOS_TOKEN = "<|endoftext|>"

EOS_TOKEN = tokenizer.eos_token 

# Read your dataset
df = pd.read_csv('/final_dataset.csv')

# Create the dictionary format
data_dict = {
    "cwe_id": df["CWE ID"].tolist(),
    "language": df["Programming Language"].tolist(),
    "code_snippet": df["Code Snippet"].tolist(),
    "vulnerable_line": df["Exact Vulnerable Line"].tolist(),
    "description": df["Description"].tolist()
}


# Split into train/validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Huggingface Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Define the formatting function exactly as in the original code
def formatting_prompts_func(examples):
    cwe_ids = examples["CWE ID"]
    languages = examples["Programming Language"]
    code_snippets = examples["Code Snippet"]
    vulnerable_lines = examples["Exact Vulnerable Line"]
    descriptions = examples["Description"]
    texts = []

    for cwe_id, language, code_snippet, vulnerable_line, description in zip(cwe_ids, languages, code_snippets, vulnerable_lines, descriptions):
        text = alpaca_prompt.format(
            cwe_id=cwe_id,
            language=language,
            code_snippet=code_snippet,
            vulnerable_line=vulnerable_line,
            description=description,
            eos=EOS_TOKEN
        )
        texts.append(text)

    return {"text": texts}

# Apply the formatting
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
val_dataset = val_dataset.map(formatting_prompts_func, batched=True)

# Preview a sample from the training dataset as the alpaca-cleaned dataset
print("\nSample from formatted training dataset:")
print(train_dataset["text"][0])


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


Sample from formatted training dataset:
You are a security expert helping developers identify vulnerable lines of code.

### Task:
Identify the exact line in the following Java code where the CWE-78 vulnerability occurs and explain briefly.

### Code:
public class Code4897 {
public int run() {
        String[] cmdList = makeCommandList();
        //L.debug(null, StringUtils.join(cmdList, " "));
        ProcessBuilder pb = new ProcessBuilder(cmdList);
        String inputDir = FilenameUtils.getFullPathNoEndSeparator(input);
        pb.directory(new File(inputDir));
        try {
            Process p = pb.start();
            return p.waitFor();
        } catch (IOException e) {
            throw new ServerException(BaseErrors.PLATFORM_IMAGE_PROCESS_ERROR, e);
        } catch (InterruptedException e) {
            throw new ServerException(BaseErrors.PLATFORM_IMAGE_PROCESS_ERROR, e);
        }
    }
}

### Output:
ProcessBuilder pb = new ProcessBuilder(cmdList);
Description: [CWE-78] U

In [6]:
print(train_dataset.column_names)

['CWE ID', 'Project Name', 'Vulnerable File', 'Programming Language', 'Line Number', 'Code Snippet', 'Exact Vulnerable Line', 'Description', '__index_level_0__', 'text']


<a name="Train"></a>
### Train the model

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # <-- Add eval dataset here
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # Switch to full training:
        #max_steps=60,
        num_train_epochs=200,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        evaluation_strategy="steps",  # <-- Enable validation
        eval_steps=100,
        save_strategy="steps",
        save_steps=200,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/9000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.063700
2,1.793500
3,1.766600
4,1.934200
5,2.214700
6,1.842800
7,1.784300
8,1.555500
9,2.118600
10,1.307000


In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')